# Block 6 Exercise 1: Non-Linear Classification

## MNIST Data
We return to the MNIST data set on handwritten digits to compare non-linear classification algorithms ...   

In [2]:
#imports 
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import fetch_openml

In [3]:
# Load data from https://www.openml.org/d/554
X, y = fetch_openml('mnist_784', version=1, return_X_y=True)


In [4]:
#the full MNIST data set contains 70k samples of digits 0-9 as 28*28 gray scale images (represented as 784 dim vectors)
np.shape(X)

(70000, 784)

In [5]:
X.min()

0.0

In [6]:
#look at max/min value in the data
X.max()

255.0

### E1.1: Cross-Validation and Support Vector Machines
Train and optimize  C-SVM classifier on MNIST (https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC)
* use a RBF kernel
* use *random search* with cross-validation to find the best settings for *gamma* and *C* (https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html#sklearn.model_selection.RandomizedSearchCV)

In [14]:
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split

In [15]:
XTrain, XTest, YTrain, YTest = train_test_split(X, y)
svm = SVC(kernel='rbf')
paramDict = dict(C=[0.1, 1, 10, 100])
search = RandomizedSearchCV(svm, paramDict, n_jobs=-1)

In [16]:
result = search.fit(XTrain, YTrain)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 4 is smaller than n_iter=10. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


In [18]:
result.best_params_


{'C': 10}

In [19]:
result.best_score_

0.9826095238095238

### E1.2: Pipelines and simple Neural Networks
Split the MNIST data into  train- and test-sets and then train and evaluate a simple Multi Layer Perceptron (MLP) network. Since the non-linear activation functions of MLPs are sensitive to the scaling on the input (recall the *sigmoid* function), we need to scale all input values to [0,1] 

* combine all steps of your training in a SKL pipeline (https://scikit-learn.org/stable/modules/compose.html#pipeline)
* use a SKL-scaler to scale the data (https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)
* MLP Parameters: https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier
    * use a *SGD* solver
    * use *tanh* as activation function
    * compare networks with 1, 2 and 3 layers, use different numbers of neurons per layer
    * adjust training parameters *alpha* (regularization) and *learning rate* - how sensitive is the model to these parameters?
    * Hint: do not change all parameters at the same time, split into several experiments
* How hard is it to find the best parameters? How many experiments would you need to find the best parameters?
    


In [6]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split

In [7]:
X, y = fetch_openml('mnist_784', version=1, return_X_y=True)
XTrain, XTest, YTrain, YTest = train_test_split(X, y)

In [53]:
#mplOneLayer = MLPClassifier(hidden_layer_sizes=(64,), activation='tanh', solver='sgd', alpha=0.0001, learning_rate_init=0.001)
mplTwoLayer = MLPClassifier(hidden_layer_sizes=(64,32,), activation='tanh', solver='sgd', alpha=0.002, learning_rate_init=0.0005)
#mplThreeLayer = MLPClassifier(hidden_layer_sizes=(64,32,16,), activation='tanh', solver='sgd', alpha=0.0001, learning_rate_init=0.001)

scaler = StandardScaler()

pipeline = make_pipeline(scaler, mplTwoLayer)

In [54]:
pipeline.fit(XTrain,YTrain)

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('mlpclassifier',
                 MLPClassifier(activation='tanh', alpha=0.002,
                               batch_size='auto', beta_1=0.9, beta_2=0.999,
                               early_stopping=False, epsilon=1e-08,
                               hidden_layer_sizes=(64, 32),
                               learning_rate='constant',
                               learning_rate_init=0.0005, max_fun=15000,
                               max_iter=200, momentum=0.9, n_iter_no_change=10,
                               nesterovs_momentum=True, power_t=0.5,
                               random_state=None, shuffle=True, solver='sgd',
                               tol=0.0001, validation_fraction=0.1,
                               verbose=False, warm_start=False))],
         verbose=False)

Scores of different Layers sizes: 

In [25]:
#OneLayer:
pipeline.score(XTest,YTest)

0.9560571428571428

In [28]:
#TwoLayers
pipeline.score(XTest,YTest)

0.9569714285714286

In [31]:
#ThreeLayers
pipeline.score(XTest,YTest)

0.9525142857142858

Alpha adjustment (with two layers): 


In [37]:
#double Alpha: 0.0002
pipeline.score(XTest,YTest)
#little bit better

0.9573714285714285

In [40]:
#double*10 Alpha:  0.002
pipeline.score(XTest,YTest)
#worse score!

0.9567428571428571

In [43]:
#times 10 alpha: 0.001
pipeline.score(XTest,YTest)
#even worse
#-> the model is pretty sensitive to changes of alpha but below the one percent mark. 

0.9564

LR adjustment (with 0.002 alpha and two layers)

In [46]:
#double LR (0.002)
pipeline.score(XTest,YTest)
#littleworst

0.9567428571428571

In [52]:
#times four LR (0.004)
pipeline.score(XTest,YTest)
#even worse

0.9557142857142857

In [55]:
#half LR (0.0005)
pipeline.score(XTest,YTest)
#worst -> the model is very sensitive to the learning rate!

0.9554857142857143